# Dynophore notebook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

from dynophores import Dynophore
from dynophores import plot, view3d

In [ ]:
dyno_path = Path("../tests/data/1KE7-1/DynophoreApp")
pdb_path = Path("../tests/data/1KE7-1/startframe.pdb")
dcd_path = Path("../tests/data/1KE7-1/trajectory.dcd")

## 3D view

In [ ]:
# Please adapt next line if your PML file is not called `dynophore.pml`
pml_path = dyno_path / "dynophore.pml"
view = view3d.show_dynophore3d(pml_path, pdb_path, dcd_path)
view.display(gui=True, style="ngl")

## Statistics

### Load data as `Dynophore` object

In [ ]:
dynophore = Dynophore.from_file(dyno_path)

### Plot interactions overview (heatmap)

In [ ]:
plot.interactive.superfeatures_vs_envpartners(dynophore);

### Plot superfeature occurrences (time series)

In [ ]:
plot.interactive.superfeatures_occurrences(dynophore);

### Plot interactions for example superfeature (time series)

#### Interaction occurrence

In [ ]:
plot.interactive.envpartners_occurrences(dynophore);

#### Interaction distances

In [ ]:
plot.interactive.envpartners_distances(dynophore);

#### Interaction profile (all-in-one)

In [ ]:
plot.interactive.envpartners_all_in_one(dynophore);